In [1]:
import pygame
import random
import math

# Dimension Definitions
SCREEN_WIDTH, SCREEN_HEIGHT = (800,500)
PLATFORM_WIDTH, PLATFORM_HEIGHT = (3000,3000)


NAME = "Agar.io"
VERSION = "1.0"

# Инициализация
pygame.init()
pygame.display.set_caption("{} - v{}".format(NAME, VERSION))
clock = pygame.time.Clock()

# Задание размеров дисплею
MAIN_SURFACE = pygame.display.set_mode((SCREEN_WIDTH,SCREEN_HEIGHT))


def getDistance(a, b):
    """ Определяет расстояние между двумя точками
    """
    diffX = math.fabs(a[0]-b[0])
    diffY = math.fabs(a[1]-b[1])
    return ((diffX**2)+(diffY**2))**(0.5)


class Paint:
    def __init__(self):
        self.paintings = []

    def add(self, col):
        self.paintings.append(col)

    def paint(self):
        for draw_elem in self.paintings:
            draw_elem.draw()


class Camera:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.H = SCREEN_HEIGHT
        self.W = SCREEN_WIDTH
        self.zoom = 0.5

    def centre(self,blobOrPos):
        """Makes sure that the given object will be at the center of player's view.
        Zoom is taken into account as well.
        """
        if isinstance(blobOrPos, Player):
            x, y = blobOrPos.x, blobOrPos.y
            self.x = (x - (x*self.zoom)) - x + (SCREEN_WIDTH/2)
            self.y = (y - (y*self.zoom)) - y + (SCREEN_HEIGHT/2)
        elif type(blobOrPos) == tuple:
            self.x, self.y = blobOrPos


    def update(self, target):
        self.zoom = 100/(target.mass)+0.3
        self.centre(player)


class Drawable:
    """Абстрактный класс для объединения дочерних классов в единый интерфейс
    """

    def __init__(self, surface, camera):
        self.surface = surface
        self.camera = camera

    def draw(self):
        pass


class Grid(Drawable):
    """Показывает сетку
    """

    def __init__(self, surface, camera):
        super().__init__(surface, camera)
        self.color = (255,255,255)

    def draw(self):
        # Совокупность горизонтальных и вертикальных линий
        zoom = self.camera.zoom
        x, y = self.camera.x, self.camera.y
        for i in range(0,2001,25):
            pygame.draw.line(self.surface,  self.color, (x, i*zoom + y), (2001*zoom + x, i*zoom + y), 2)
            pygame.draw.line(self.surface, self.color, (i*zoom + x, y), (i*zoom + x, 2001*zoom + y), 2)


class Cell(Drawable):
    """Генерация порций масс
    """
    CELL_COLORS = [
    (255, 250, 210),
    (36,254,255),
    (233,21,46),
    (4,39,203),
    (254,6,178),
    (205,211,3),
    (216,6,254),
    (85,252,50),
    (145,255,2),
    (3,255,182),
    (255,8,46),
    (142,18,255),
    (0, 0, 0)]

    def __init__(self, surface, camera):
        super().__init__(surface, camera)
        self.x = random.randint(20,1980)
        self.y = random.randint(20,1980)
        self.mass = 7
        self.color = random.choice(Cell.CELL_COLORS)

    def draw(self):
        """Отрисовка порции массы
        """
        zoom = self.camera.zoom
        x,y = self.camera.x, self.camera.y
        center = (int(self.x*zoom + x), int(self.y*zoom + y))
        pygame.draw.circle(self.surface, self.color, center, int(self.mass*zoom))


class CellList(Drawable):
    """Для списка частиц и их количества на экране
    """

    def __init__(self, surface, camera, numb_cells):
        super().__init__(surface, camera)
        self.count = numb_cells
        self.list = []
        for i in range(self.count):
            self.list.append(Cell(self.surface, self.camera))

    def draw(self):
        for cell in self.list:
                cell.draw()


class Player(Drawable):
    def __init__(self, surface, camera):
        super().__init__(surface, camera)
        self.x = random.randint(100,400)
        self.y = random.randint(100,400)
        self.mass = 40
        self.speed = 5
        self.color = col = (255, 255, 255)
        self.outlineColor = (
            int(col[0]-col[0]/3),
            int(col[1]-col[1]/3),
            int(col[2]-col[2]/3))


    def collisionDetection(self, feeds):
        """Проверка на столкновение нашего шара с едой
        """
        for feed in feeds:
            if(getDistance((feed.x, feed.y), (self.x,self.y)) <= self.mass/2):
                if feed.color == (0, 0, 0):
                    self.mass += 6
                else:
                    self.mass += 0.6

                feeds.remove(feed)


    def move(self):
        """ Изменение позиции игрока, относительно координат его мыши
        """
        dX, dY = pygame.mouse.get_pos()
        # Находит угол от центра экрана к мышке (в радианах)
        rotation = math.atan2(dY - float(SCREEN_HEIGHT)/2, dX - float(SCREEN_WIDTH)/2)
        # Конвертация радиан в градусную меру углов
        rotation *= 180/math.pi
        # Нормализация
        normalized = (90 - math.fabs(rotation))/90
        vx = self.speed*normalized
        vy = 0
        if rotation < 0:
            vy = -self.speed + math.fabs(vx)
        else:
            vy = self.speed - math.fabs(vx)
        tmpX = self.x + vx
        tmpY = self.y + vy
        self.x = tmpX
        self.y = tmpY


    def speedrun(self):
        self.speed += 1


    def unspeedrun(self):
        self.speed -= 1


    def draw(self):
        """Рисует игрока
        """
        zoom = self.camera.zoom
        x, y = self.camera.x, self.camera.y
        center = (int(self.x*zoom + x), int(self.y*zoom + y))

        # Рисует внешню границу круга игрока
        pygame.draw.circle(self.surface, self.outlineColor, center, int((self.mass/2 + 3)*zoom))
        # Рисует реальную площадь игрока
        pygame.draw.circle(self.surface, self.color, center, int(self.mass/2*zoom))

# Инициализация сущностей (камера, сетка, ячейки, и т.д.)
cam = Camera()

grid = Grid(MAIN_SURFACE, cam)
cells = CellList(MAIN_SURFACE, cam, 1500)
player = Player(MAIN_SURFACE, cam)

# Объект для рисования
painter = Paint()

# Добавляем элементы в список для отрисовки
painter.add(grid)
painter.add(cells)
painter.add(player)


# Игровой цикл
while True:
    clock.tick(80)  # частота кадров не превышает 80

    #   Проверка на действия (события) от игрока
    for event in pygame.event.get():
        if(event.type == pygame.KEYDOWN):
            # Закрытие программы
            if(event.key == pygame.K_ESCAPE):
                pygame.quit()
                quit()
            if(event.key == pygame.K_s):
                player.speedrun()
            if(event.key == pygame.K_a):
                player.unspeedrun()
        # Закрытие программы
        if(event.type == pygame.QUIT):
            pygame.quit()
            quit()

    player.move()
    player.collisionDetection(cells.list)
    cam.update(player)
    MAIN_SURFACE.fill((40,40,40))  # Темная тема
    painter.paint()  # Отрисовка
    # Запуск следующего вычисления дисплея
    pygame.display.flip()


pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: video system not initialized